Data Cleaning

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [4]:
df=pd.read_excel("Losses.xlsx")

In [5]:
df['DATE'] = df[['DAY','MONTH_ID','YEAR']].apply(lambda x: '-'.join(x.values.astype(str)), axis="columns")

In [11]:
df['DATE']=pd.to_datetime(df['DATE'],format='mixed',dayfirst=True)

In [16]:
df.drop(df[['DAY','MONTH','MONTH_ID','YEAR']],axis=1,inplace=True)

In [19]:
df.set_index("DATE", inplace = True)

In [20]:
df.to_clipboard()

Import updated Data

In [23]:
df=pd.read_excel("Losses.xlsx",sheet_name='Sheet1',index_col='DATE')

In [25]:
df.head()

,CAUSE,GROSS INCURRED AMOUNT
DATE,,
1999-01-01,WINDSTORM,477.88
1999-01-01,FIRE,700.00
1999-01-01,WINDSTORM,99.87
1999-01-01,WINDSTORM,139.80
1999-01-01,WINDSTORM,548.66


Preprocessing

In [28]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.statespace.varmax import VARMAX
from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import grangercausalitytests, adfuller

In [29]:
d={'WINDSTORM': 0,
 'FIRE': 1,
 'ESCAPE OF WATER': 2,
 'SUBSIDENCE     ': 3,
 'FLOOD          ': 4,
 'ACCIDENTAL DAMAGE': 5,
 'THEFT': 6,
 'EARTHQUAKE     ': 7}

In [30]:
df['CAUSE']=df['CAUSE'].map(d)

In [39]:
ad_fuller_result_1 = adfuller(df['GROSS INCURRED AMOUNT'].diff()[1:])

In [44]:
ad_fuller_result_1 = adfuller(df['GROSS INCURRED AMOUNT'])

print('GROSS INCURRED AMOUNT')
print(f'ADF Statistic: {ad_fuller_result_1[0]}')
print(f'p-value: {ad_fuller_result_1[1]}')

print('\n---------------------\n')

ad_fuller_result_2 = adfuller(df['CAUSE'])

print('CAUSE')
print(f'ADF Statistic: {ad_fuller_result_2[0]}')
print(f'p-value: {ad_fuller_result_2[1]}')

GROSS INCURRED AMOUNT
ADF Statistic: -20.689410184199218
p-value: 0.0

---------------------

CAUSE
ADF Statistic: -10.695166657819412
p-value: 3.640459606781284e-19


In [42]:
print('GROSS INCURRED AMOUNT causes CAUSE?\n')
print('------------------')
granger_1 = grangercausalitytests(df[['CAUSE', 'GROSS INCURRED AMOUNT']], 4)

print('\nCAUSE causes GROSS INCURRED AMOUNT?\n')
print('------------------')
granger_2 = grangercausalitytests(df[['GROSS INCURRED AMOUNT', 'CAUSE']], 4)

GROSS INCURRED AMOUNT causes CAUSE?

------------------

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=2.5722  , p=0.1088  , df_denom=47561, df_num=1
ssr based chi2 test:   chi2=2.5724  , p=0.1087  , df=1
likelihood ratio test: chi2=2.5723  , p=0.1088  , df=1
parameter F test:         F=2.5722  , p=0.1088  , df_denom=47561, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=5.4608  , p=0.0043  , df_denom=47558, df_num=2
ssr based chi2 test:   chi2=10.9228 , p=0.0042  , df=2
likelihood ratio test: chi2=10.9215 , p=0.0043  , df=2
parameter F test:         F=5.4608  , p=0.0043  , df_denom=47558, df_num=2

Granger Causality
number of lags (no zero) 3
ssr based F test:         F=3.7567  , p=0.0104  , df_denom=47555, df_num=3
ssr based chi2 test:   chi2=11.2717 , p=0.0103  , df=3
likelihood ratio test: chi2=11.2703 , p=0.0104  , df=3
parameter F test:         F=3.7567  , p=0.0104  , df_denom=47555, df_num=3

Granger Causality
number 

In [46]:
train_df=df[:-12]
test_df=df[-12:]

In [47]:
model = VAR(train_df)

C:\Users\Rumit\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


In [51]:
sorted_order=model.select_order(maxlags=20)
print(sorted_order.summary())

 VAR Order Selection (* highlights the minimums)  
       AIC         BIC         FPE         HQIC   
--------------------------------------------------
0        18.17       18.17   7.816e+07       18.17
1        18.03       18.03   6.743e+07       18.03
2        17.98       17.98   6.440e+07       17.98
3        17.95       17.95   6.257e+07       17.95
4        17.93       17.93   6.129e+07       17.93
5        17.92       17.92   6.036e+07       17.92
6        17.90       17.91   5.964e+07       17.91
7        17.89       17.90   5.907e+07       17.90
8        17.89       17.89   5.867e+07       17.89
9        17.88       17.89   5.831e+07       17.88
10       17.88       17.88   5.803e+07       17.88
11       17.87       17.88   5.789e+07       17.88
12       17.87       17.88   5.770e+07       17.87
13       17.87       17.88   5.755e+07       17.87
14       17.87       17.88   5.743e+07       17.87
15       17.86       17.87   5.726e+07       17.87
16       17.86       17.87   5.

In [52]:
var_model = VARMAX(train_df, order=(4,0),enforce_stationarity= True)
fitted_model = var_model.fit(disp=False)
print(fitted_model.summary())

C:\Users\Rumit\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


                                   Statespace Model Results                                   
Dep. Variable:     ['CAUSE', 'GROSS INCURRED AMOUNT']   No. Observations:                47553
Model:                                         VAR(4)   Log Likelihood             -561283.176
                                          + intercept   AIC                        1122608.352
Date:                                Wed, 26 Apr 2023   BIC                        1122792.514
Time:                                        22:06:52   HQIC                       1122666.175
Sample:                                             0                                         
                                              - 47553                                         
Covariance Type:                                  opg                                         
Ljung-Box (L1) (Q):            13.26, 0.06   Jarque-Bera (JB):   2731.66, 1406251783.99
Prob(Q):                        0.00, 0.81   Prob(JB):   

In [53]:
n_forecast = 12
predict = fitted_model.get_prediction(start=len(train_df),end=len(train_df) + n_forecast-1)
predictions=predict.predicted_mean

C:\Users\Rumit\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\base\tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(


In [54]:
predictions.columns=['Causes','Gross']
predictions

,Causes,Gross
47553,3.900453,904.763608
47554,3.688894,1049.639541
47555,3.477792,1090.334875
47556,3.802650,1060.537626
47557,3.664223,1058.471641
47558,3.623744,1071.872050
47559,3.609906,1073.569398
47560,3.627853,1073.205454
47561,3.605093,1071.966006
47562,3.595397,1073.040723


In [64]:
test_df

,CAUSE,GROSS INCURRED AMOUNT
DATE,,
2007-12-27,1,2251.0
2007-12-27,5,461.0
2007-12-28,4,1051.0
2007-12-28,1,2251.0
2007-12-28,5,461.0
2007-12-28,0,901.0
2007-12-28,6,1151.0
2007-12-28,4,1051.0
2007-12-29,5,461.0


In [70]:
from sklearn.metrics import mean_squared_error
import math 
from statistics import mean

CAUSE=math.sqrt(mean_squared_error(predictions['Causes'],test_df['CAUSE']))
print('Mean value of CAUSE is : {}. Root Mean Squared Error is :{}'.format(mean(test_df['CAUSE']),CAUSE))

Gross=math.sqrt(mean_squared_error(predictions['Gross'],test_df['GROSS INCURRED AMOUNT']))
print('Mean value of Gross is : {}. Root Mean Squared Error is :{}'.format(mean(test_df['GROSS INCURRED AMOUNT']),Gross))

Mean value of CAUSE is : 3.6666666666666665. Root Mean Squared Error is :1.8995106277703833
Mean value of Gross is : 1050.1666666666667. Root Mean Squared Error is :627.0920534008175
